Import the libraries needed

In [14]:
import pandas as pd
import numpy as np
import os
import pyodbc
pyodbc.drivers()

from datetime import datetime
import psutil

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

Accessing the legacy Oracle database through a SQL Server linked server.  This is to cut down on intergration points.  The recent switch data will exist on SQl Server and a linked server exists on the same instance to the legacy Oracle data.

### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'WINPRDAI1572\CLESQLDW'
Reporting_database = 'CLE_Reporting'

pd.set_option('max_row', None)

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

filename = input_path + "\iSuite_Product_list.csv"
print(filename)

.\data\iSuite_Product_list.csv


# 1. Get all iSuite policies that can have switchable funds.
## - Start by determining what's the population of all policies that can possibly have a switch.
##  - Get a list of products that have had a switch at some point.  
## - This allows us to look for a list of all policies that can ever have, or may have had, a fund switch.
## - This determines the list of products that allows us to determine the full switchable population at any time by identifying policies in force for any of the products found.

In [3]:
# READ IN directly from DW
def iSuite_Product_list(server, database):
    conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+server+';'
         'DATABASE='+database+';'
         'Trusted_Connection=yes;')
  
    query = """SELECT DISTINCT pol.Product_Code
		    FROM [CLE_Reporting].[user].[CLV_GBS_Unit_Transactions] UT
	        INNER JOIN CLE_Reporting.[user].CLV_Fund F ON F.Fund_Code = UT.[Fund_Code]
	        INNER JOIN CLE_Reporting.dbo.CLT_Policy pol ON pol.Policy_Number = UT.Policy_Number
			WHERE Transaction_Type = 'Funds Switch/Shift'
			AND	((F.Asset_Type = 'UWP-Fonds' AND UWP_Type_Indicator = 'S') OR (F.Asset_Type <> 'UWP-Fonds' AND UWP_Type_Indicator = 'A')) 
			AND Reverse_Indicator = 'N'"""    
        
    return pd.read_sql(query, conn)

# Call the SQL proc to get a list of distinct iSuite product codes, 
# of products that have ever had a switch.
df_iSuite_Product_list = iSuite_Product_list(SQL_server, Reporting_database)

# Strip all blanks.
df_iSuite_Product_list = df_iSuite_Product_list.applymap(lambda x: " ".join(x.split()) if isinstance(x, str) else x)

df_iSuite_Product_list.head(10)

# Write out to a comma separated values file.
filename = input_path + "\iSuite_Product_list.csv"
print(filename)

df_iSuite_Product_list.to_csv(filename, encoding='utf-8', index=False)        
df_iSuite_Product_list.head(5)

.\data\iSuite_Product_list.csv


,Product_Code
0,FKP_2006V1
1,FKP_2006V2
2,FKP_2008V1
3,FKP_2010V1
4,FKP_2012V1


# 2. Using the list of products which have had a switch, get details for all policies that can have switchable funds.
## This is the population of policies that can possibly switch funds.

In [4]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

main_query = f"""SELECT pol.Policy_Number         AS ContractNumber
                       ,prd.Product_Category_Code AS ProdCat
                       ,prd.Product_Code          AS ProdCode
                       ,psh.Policy_Status_Code    AS PolicyStatus
                       ,pol.Cover_End_Date        AS CoverEndDt
                       ,psh.Snapshot_Date         AS SnapshotDt
                 FROM CLT_Policy pol
                 INNER JOIN CLE_Reporting.dbo.CLT_Product prd ON prd.Product_Id = pol.Product_Id
                 INNER JOIN CLT_Policy_Snapshot_Hist psh ON pol.Policy_Number = psh.Policy_Number 
                 WHERE prd.Product_Id = pol.Product_Id
                 AND   prd.Product_Code IN ('FKP_2006V1','FKP_2006V2','FKP_2008V1','FKP_2010V1','FKP_2012V1','FKP_2017V1',
                                            'GNPP_2016V1','GNPP_2017V1','GNPP_2022V1','GNPP_2023V1',
                                            'GNRP_2008V1','GNRP_2010V1','GNRP_2012V1','GNRP_2017V1','GNRP_2022V1',
                                            'XRPP_2005V1','XRPP_2008V1','XRPP_2010V1','XRPP_2012V1','XRPP_2015V1')
                 ORDER BY psh.Snapshot_Date,pol.Policy_Number;"""



df_main = pd.read_sql(main_query, conn)

df_main['SnapshotDt'] = pd.to_datetime(df_main['SnapshotDt'])
df_main['SnapshotYrMth'] = df_main['SnapshotDt'].dt.strftime('%Y%m')
#df_main = df_main.drop(['SnapshotDt'],axis=1)

# Types and row count
print(len(df_main.index))
print(df_main.dtypes)

# Write out to a comma separated values file.
filename = input_path + '\Full_Population.csv'
df_main.to_csv(filename, encoding='utf-8', index=False)        
print(filename)
df_main.head(10)

8411354
ContractNumber            object
ProdCat                   object
ProdCode                  object
PolicyStatus              object
CoverEndDt                object
SnapshotDt        datetime64[ns]
SnapshotYrMth             object
dtype: object


,ContractNumber,ProdCat,ProdCode,PolicyStatus,CoverEndDt,SnapshotDt,SnapshotYrMth
0,6074160W,XRPP,XRPP_2005V1,I,2028-12-01,2016-07-31,201607
1,6074937L,XRPP,XRPP_2005V1,I,2033-12-01,2016-07-31,201607
2,6074962M,XRPP,XRPP_2005V1,I,2042-12-01,2016-07-31,201607
3,6075850C,XRPP,XRPP_2005V1,I,2034-12-01,2016-07-31,201607
4,6100638P,XRPP,XRPP_2005V1,I,2046-04-01,2016-07-31,201607
5,6100746E,XRPP,XRPP_2005V1,F,2043-05-01,2016-07-31,201607
6,6214246Y,XRPP,XRPP_2005V1,I,2034-10-01,2016-07-31,201607
7,6215630F,XRPP,XRPP_2005V1,I,2042-12-01,2016-07-31,201607
8,6215740Y,XRPP,XRPP_2005V1,I,2034-10-01,2016-07-31,201607
9,6215812A,XRPP,XRPP_2005V1,I,2045-10-01,2016-07-31,201607


# 3. Get the fund switch data.

## Get the Paxus fund switches¶

In [5]:
# READ IN directly from Paxus
def Get_All_Paxus_Switches(server, database):
    conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+server+';'
         'DATABASE='+database+';'
         'Trusted_Connection=yes;'
         )

    query = """SELECT * FROM OPENQUERY(GERPDB,'
    SELECT  p.policy            AS "POLICY_NUMBER",
            u.fund              AS "LEGACY_FUND_CODE",
            p.cov_cobe          AS "PRODUCT",
            to_date(u.proc_date,''YYYYMMDD'') AS "PROCESSING_DATE",
            to_date(u.eff_date,''YYYYMMDD'')  AS "EFFECTIVE_DATE",
            u.unit_type                       AS "UNIT_TYPE",
            u.charge_type                     AS "CHARGE_TYPE",
            u.units_posted_cash               AS "CASH_VALUE",
            to_date(p.rcd,''YYYYMMDD'')       AS "RISK_COMMENCEMENT_DATE",
            p.agent                             AS "Servicing_Broker_Code",
            to_date(p.issued_date,''YYYYMMDD'') AS "Issue_Date",
            to_date(''19000101'',''YYYYMMDD'')  AS "Cover_End_Date",
            to_date(''19000101'',''YYYYMMDD'')  AS "Anniversary_Date",
            ''P''                               AS "Source"
    FROM GERPDBA.UNIT u,gerpdba.policy p 
    WHERE p.policy = u.policy 
    AND u.trans_code = ''P135'' 
    AND to_date(u.proc_date,''YYYYMMDD'') > ''2018-12-31''
    order by PROC_DATE, p.policy, u.fund
    ')"""    
    
    
    return pd.read_sql(query, conn)

# Call the SQL process to retrieve the Paxus (old system) fund switch data.
df_All_Paxus_Switches = Get_All_Paxus_Switches(SQL_server, Reporting_database)

# Types and row count
print(len(df_All_Paxus_Switches.index))
print(df_All_Paxus_Switches.dtypes)

# Write out to a comma separated values file.
filename = input_path + '\All_Paxus_Switches.csv'
df_All_Paxus_Switches.to_csv(filename, encoding='utf-8', index=False)        
print(filename)
df_All_Paxus_Switches.head(10)

183962
POLICY_NUMBER                     object
LEGACY_FUND_CODE                  object
PRODUCT                           object
PROCESSING_DATE           datetime64[ns]
EFFECTIVE_DATE            datetime64[ns]
UNIT_TYPE                         object
CHARGE_TYPE                       object
CASH_VALUE                       float64
RISK_COMMENCEMENT_DATE    datetime64[ns]
Servicing_Broker_Code             object
Issue_Date                datetime64[ns]
Cover_End_Date            datetime64[ns]
Anniversary_Date          datetime64[ns]
Source                            object
dtype: object


,POLICY_NUMBER,LEGACY_FUND_CODE,PRODUCT,PROCESSING_DATE,EFFECTIVE_DATE,UNIT_TYPE,CHARGE_TYPE,CASH_VALUE,RISK_COMMENCEMENT_DATE,Servicing_Broker_Code,Issue_Date,Cover_End_Date,Anniversary_Date,Source
0,6535239K,XMLEM,B2CO,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P
1,6535239K,XSK10,B2CO,2019-01-02,2018-12-28,A,B,-5527.4800,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P
2,6535239K,XSKN3,B2CO,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P
3,6562520M,XSDIV,B2AO,2019-01-02,2018-12-27,A,B,10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,P
4,6562520M,XSEDD,B2AO,2019-01-02,2018-12-27,A,B,-10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,P
5,6651310C,BUWP6,C11A,2019-01-02,2018-12-31,A,B,-221.3397,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
6,6651310C,RUWP6,C11A,2019-01-02,2018-12-31,A,B,-231.0452,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
7,6651310C,XUEAA,C11A,2019-01-02,2018-12-31,A,B,-212.5059,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
8,6651310C,XUEAA,C11A,2019-01-02,2018-12-31,A,B,433.2002,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
9,6128381Y,SUEAM,PXCF,2019-01-03,2019-01-02,A,B,-327.4962,2004-11-01,E396,2004-10-22,1900-01-01,1900-01-01,P


# 2. Get the iSuite fund switches.

In [6]:
def Get_All_iSuite_Switches(server, database):
    conn = pyodbc.connect(        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER='+server+';'
        'DATABASE='+database+';'
        'Trusted_Connection=yes;'  )

    query_201 = """Select 
	FS_List.Policy_Number         AS "POLICY_NUMBER",
	Legacy_Fund_Code              AS "LEGACY_FUND_CODE",
    PC.Product_Code               AS "PRODUCT",
	FS_List.Processing_Date       AS "PROCESSING_DATE",
	FS_List.Effective_Date        AS "EFFECTIVE_DATE",
	FS_List.Unit_Type             AS "UNIT_TYPE",
    FS_List.Charge_Type           AS "CHARGE_TYPE",
    Cash_Value                    AS "CASH_VALUE",
	PC.Risk_Commencement_Date     AS "RISK_COMMENCEMENT_DATE",
    Servicing_Broker_Code,
	Issue_Date,
    Cover_End_Date,
    Anniversary_Date,
    'I'                         AS "Source"

FROM [CLE_Reporting].[user].[CLV_Policy_Core] PC 
INNER JOIN (SELECT Policy_Number,
		           Transaction_Type,
				   UT.Processing_Date,
				   UT.Effective_Date,
				   UT.Unit_Type, 
				   UT.Charge_Type,                    
				   Cash_Value,
				   F.Fund_Code,
				   F.Legacy_Fund_Code
		    FROM [CLE_Reporting].[user].[CLV_GBS_Unit_Transactions] UT
	        INNER JOIN CLE_Reporting.[user].CLV_Fund F 
			ON F.Fund_Code = UT.[Fund_Code]
			WHERE Transaction_Type = 'Funds Switch/Shift'
			AND	((F.Asset_Type = 'UWP-Fonds' AND UWP_Type_Indicator = 'S') OR (F.Asset_Type <> 'UWP-Fonds' AND UWP_Type_Indicator = 'A')) 
			AND Reverse_Indicator = 'N') FS_LIST
ON FS_LIST.[Policy_Number] = PC.Policy_Number
WHERE FS_List.Processing_Date > '2018-12-31'
ORDER BY Processing_Date,Policy_Number
    """
    return pd.read_sql(query_201, conn)

# Call the SQL process to retrieve the Paxus (old system) fund switch data.
df_All_iSuite_Switches = Get_All_iSuite_Switches(SQL_server, Reporting_database)

# Types and row count
print(len(df_All_iSuite_Switches.index))
print(df_All_iSuite_Switches.dtypes)

# Write out to a comma separated values file.
filename = input_path + '\All_iSuite_Switches.csv'
df_All_iSuite_Switches.to_csv(filename, encoding='utf-8', index=False)      
print(filename)
df_All_iSuite_Switches.head(10)

17843
POLICY_NUMBER              object
LEGACY_FUND_CODE           object
PRODUCT                    object
PROCESSING_DATE            object
EFFECTIVE_DATE             object
UNIT_TYPE                  object
CHARGE_TYPE                object
CASH_VALUE                float64
RISK_COMMENCEMENT_DATE     object
Servicing_Broker_Code      object
Issue_Date                 object
Cover_End_Date             object
Anniversary_Date           object
Source                     object
dtype: object


,POLICY_NUMBER,LEGACY_FUND_CODE,PRODUCT,PROCESSING_DATE,EFFECTIVE_DATE,UNIT_TYPE,CHARGE_TYPE,CASH_VALUE,RISK_COMMENCEMENT_DATE,Servicing_Broker_Code,Issue_Date,Cover_End_Date,Anniversary_Date,Source
0,8016254J,XSETH,FKP_2017V1,2020-03-16,2020-03-12,A,BID,-2816.34,2020-02-12,7192,2020-02-10,2050-02-12,2020-02-12,I
1,8016254J,XSGRN,FKP_2017V1,2020-03-16,2020-03-12,A,BID,-3223.49,2020-02-12,7192,2020-02-10,2050-02-12,2020-02-12,I
2,8016254J,XSECM,FKP_2017V1,2020-03-16,2020-03-12,A,BID,39321.07,2020-02-12,7192,2020-02-10,2050-02-12,2020-02-12,I
3,8016254J,XSDIV,FKP_2017V1,2020-03-16,2020-03-12,A,BID,-7111.82,2020-02-12,7192,2020-02-10,2050-02-12,2020-02-12,I
4,8016254J,XSEBM,FKP_2017V1,2020-03-16,2020-03-12,A,BID,-26169.42,2020-02-12,7192,2020-02-10,2050-02-12,2020-02-12,I
5,8015490J,XSDIT,FKP_2017V1,2020-04-29,2020-03-24,A,BID,9808.89,2020-03-20,B86X,2020-03-18,2075-03-20,2020-03-20,I
6,8015490J,XSSIT,FKP_2017V1,2020-04-29,2020-03-24,A,BID,10022.82,2020-03-20,B86X,2020-03-18,2075-03-20,2020-03-20,I
7,8015490J,XSEDD,FKP_2017V1,2020-04-29,2020-03-24,A,BID,-48672.03,2020-03-20,B86X,2020-03-18,2075-03-20,2020-03-20,I
8,8015490J,XSEIE,FKP_2017V1,2020-04-29,2020-03-24,A,BID,20045.64,2020-03-20,B86X,2020-03-18,2075-03-20,2020-03-20,I
9,8015490J,XSECM,FKP_2017V1,2020-04-29,2020-03-24,A,BID,-4498.92,2020-03-20,B86X,2020-03-18,2075-03-20,2020-03-20,I


# Convert Paxus Cobes to Product Name

In [7]:
def Get_Product_Mapping(server, database):
    conn = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER='+server+';'
        'DATABASE='+database+';'
        'Trusted_Connection=yes;'
        )

    query_201 = "Select * FROM CLT_Product_Mapping"
    return pd.read_sql(query_201, conn)

# Call the SQL to get Paxus to iSuite product mappings.
df_Product_Mapping = Get_Product_Mapping(SQL_server, Reference_Data_database)  

# Strip all blanks.
df_Product_Mapping = df_Product_Mapping.applymap(lambda x: " ".join(x.split()) if isinstance(x, str) else x)
df_Product_Mapping.head(99)

,Cobe_Code,Product_Code
0,B1AF,GNP_2012V1
1,B1AO,GNP_2012V1
2,B1BF,GNP_2012V1
3,B1BO,GNP_2012V1
4,B1CF,GNP_2012V1
5,B1CO,GNP_2012V1
6,B1DF,GNP_2012V1
7,B1EF,GNP_2012V1
8,B1FF,GNP_2012V1
9,B1GF,GNP_2012V1


## Write out to a comma separated values file.

In [8]:
# Write out to a comma separated values file.
filename = input_path + '\df_Product_Mapping.csv'
df_Product_Mapping.to_csv(filename, encoding='utf-8', index=False)       
print(filename)
df_Product_Mapping.head(10)

,Cobe_Code,Product_Code
0,B1AF,GNP_2012V1
1,B1AO,GNP_2012V1
2,B1BF,GNP_2012V1
3,B1BO,GNP_2012V1
4,B1CF,GNP_2012V1
5,B1CO,GNP_2012V1
6,B1DF,GNP_2012V1
7,B1EF,GNP_2012V1
8,B1FF,GNP_2012V1
9,B1GF,GNP_2012V1


## Join the Paxus switch to teh Paxus product mapping to make it have the same mapping as iSuite.

In [9]:
# Strip the blanks from the product to allow them to join.
df_All_Paxus_Switches['PRODUCT'] = df_All_Paxus_Switches['PRODUCT'].str.strip()

# Join them
df_All_Paxus_Switches_With_Product_Name = pd.merge(df_All_Paxus_Switches, df_Product_Mapping, left_on = ['PRODUCT'], right_on = ['Cobe_Code'] , how = 'left') 
df_All_Paxus_Switches_With_Product_Name.head(5)

,POLICY_NUMBER,LEGACY_FUND_CODE,PRODUCT,PROCESSING_DATE,EFFECTIVE_DATE,UNIT_TYPE,CHARGE_TYPE,CASH_VALUE,RISK_COMMENCEMENT_DATE,Servicing_Broker_Code,Issue_Date,Cover_End_Date,Anniversary_Date,Source,Cobe_Code,Product_Code
0,6535239K,XMLEM,B2CO,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P,B2CO,GNRP_2012V1
1,6535239K,XSK10,B2CO,2019-01-02,2018-12-28,A,B,-5527.4800,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P,B2CO,GNRP_2012V1
2,6535239K,XSKN3,B2CO,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P,B2CO,GNRP_2012V1
3,6562520M,XSDIV,B2AO,2019-01-02,2018-12-27,A,B,10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,P,B2AO,GNRP_2012V1
4,6562520M,XSEDD,B2AO,2019-01-02,2018-12-27,A,B,-10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,P,B2AO,GNRP_2012V1


## Clean the data and write to file.

In [10]:
# Remove unused columns after merge.
df_All_Paxus_Switches_With_Product_Name = df_All_Paxus_Switches_With_Product_Name.drop(['PRODUCT','Cobe_Code'], axis=1)

# Rename columns to match.
df_All_Paxus_Switches_With_Product_Name = df_All_Paxus_Switches_With_Product_Name.rename(columns={"Product_Code":"PRODUCT"})
df_All_Paxus_Switches_With_Product_Name = df_All_Paxus_Switches_With_Product_Name.rename(columns={"Source"      :"SOURCE"})

# Reorder the data before the merge.
neworder = ['POLICY_NUMBER','LEGACY_FUND_CODE','PRODUCT','PROCESSING_DATE','EFFECTIVE_DATE','UNIT_TYPE','CHARGE_TYPE','CASH_VALUE','RISK_COMMENCEMENT_DATE','Servicing_Broker_Code','Issue_Date','Cover_End_Date','Anniversary_Date','SOURCE']
df_All_Paxus_Switches_With_Product_Name=df_All_Paxus_Switches_With_Product_Name.reindex(columns=neworder)

# Example 1: Convert datetype to string
df_All_Paxus_Switches_With_Product_Name['PROCESSING_DATE']=df_All_Paxus_Switches_With_Product_Name['PROCESSING_DATE'].astype(str)
df_All_Paxus_Switches_With_Product_Name['EFFECTIVE_DATE']=df_All_Paxus_Switches_With_Product_Name['EFFECTIVE_DATE'].astype(str)
df_All_Paxus_Switches_With_Product_Name['RISK_COMMENCEMENT_DATE']=df_All_Paxus_Switches_With_Product_Name['RISK_COMMENCEMENT_DATE'].astype(str)
df_All_Paxus_Switches_With_Product_Name['Issue_Date']=df_All_Paxus_Switches_With_Product_Name['Issue_Date'].astype(str)
df_All_Paxus_Switches_With_Product_Name['Cover_End_Date']=df_All_Paxus_Switches_With_Product_Name['Cover_End_Date'].astype(str)
df_All_Paxus_Switches_With_Product_Name['Anniversary_Date']=df_All_Paxus_Switches_With_Product_Name['Anniversary_Date'].astype(str)


# Types and row count
print(len(df_All_Paxus_Switches_With_Product_Name.index))
print(df_All_Paxus_Switches_With_Product_Name.dtypes)

# Write out to a comma separated values file.
filename = input_path + '\All_Paxus_Switches_With_Product_Name.csv'
print(filename)
df_All_Paxus_Switches_With_Product_Name.to_csv(filename, encoding='utf-8', index=False)        
df_All_Paxus_Switches_With_Product_Name.head(10)


183962
POLICY_NUMBER              object
LEGACY_FUND_CODE           object
PRODUCT                    object
PROCESSING_DATE            object
EFFECTIVE_DATE             object
UNIT_TYPE                  object
CHARGE_TYPE                object
CASH_VALUE                float64
RISK_COMMENCEMENT_DATE     object
Servicing_Broker_Code      object
Issue_Date                 object
Cover_End_Date             object
Anniversary_Date           object
SOURCE                     object
dtype: object


,POLICY_NUMBER,LEGACY_FUND_CODE,PRODUCT,PROCESSING_DATE,EFFECTIVE_DATE,UNIT_TYPE,CHARGE_TYPE,CASH_VALUE,RISK_COMMENCEMENT_DATE,Servicing_Broker_Code,Issue_Date,Cover_End_Date,Anniversary_Date,SOURCE
0,6535239K,XMLEM,GNRP_2012V1,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P
1,6535239K,XSK10,GNRP_2012V1,2019-01-02,2018-12-28,A,B,-5527.4800,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P
2,6535239K,XSKN3,GNRP_2012V1,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,P
3,6562520M,XSDIV,GNRP_2012V1,2019-01-02,2018-12-27,A,B,10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,P
4,6562520M,XSEDD,GNRP_2012V1,2019-01-02,2018-12-27,A,B,-10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,P
5,6651310C,BUWP6,GNPP_2016V1,2019-01-02,2018-12-31,A,B,-221.3397,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
6,6651310C,RUWP6,GNPP_2016V1,2019-01-02,2018-12-31,A,B,-231.0452,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
7,6651310C,XUEAA,GNPP_2016V1,2019-01-02,2018-12-31,A,B,-212.5059,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
8,6651310C,XUEAA,GNPP_2016V1,2019-01-02,2018-12-31,A,B,433.2002,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,P
9,6128381Y,SUEAM,XRP_2003V2,2019-01-03,2019-01-02,A,B,-327.4962,2004-11-01,E396,2004-10-22,1900-01-01,1900-01-01,P


# Merge all switches into one dataframe and write to file.

In [11]:
# Merge the Paxus and iSuite switch data.
df_All_Switches= df_All_iSuite_Switches.append(df_All_Paxus_Switches_With_Product_Name)

# Soryt by policy number and processing date.
df_All_Switches = df_All_Switches.sort_values(['PROCESSING_DATE','POLICY_NUMBER','SOURCE']).reset_index(drop=False)

# Types and row count
print(len(df_All_Switches.index))
print(df_All_Switches.dtypes)

# Write out to a comma separated values file.
filename = input_path + '\All_Switches.csv'
df_All_Switches.to_csv(filename, encoding='utf-8', index=False)       
print(filename)
df_All_Switches.head(10)

201805
index                       int64
POLICY_NUMBER              object
LEGACY_FUND_CODE           object
PRODUCT                    object
PROCESSING_DATE            object
EFFECTIVE_DATE             object
UNIT_TYPE                  object
CHARGE_TYPE                object
CASH_VALUE                float64
RISK_COMMENCEMENT_DATE     object
Servicing_Broker_Code      object
Issue_Date                 object
Cover_End_Date             object
Anniversary_Date           object
Source                     object
SOURCE                     object
dtype: object


,index,POLICY_NUMBER,LEGACY_FUND_CODE,PRODUCT,PROCESSING_DATE,EFFECTIVE_DATE,UNIT_TYPE,CHARGE_TYPE,CASH_VALUE,RISK_COMMENCEMENT_DATE,Servicing_Broker_Code,Issue_Date,Cover_End_Date,Anniversary_Date,Source,SOURCE
0,0,6535239K,XMLEM,GNRP_2012V1,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,NaN,P
1,1,6535239K,XSK10,GNRP_2012V1,2019-01-02,2018-12-28,A,B,-5527.4800,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,NaN,P
2,2,6535239K,XSKN3,GNRP_2012V1,2019-01-02,2018-12-28,A,B,2763.7400,2013-12-19,A28E,2013-12-19,1900-01-01,1900-01-01,NaN,P
3,3,6562520M,XSDIV,GNRP_2012V1,2019-01-02,2018-12-27,A,B,10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,NaN,P
4,4,6562520M,XSEDD,GNRP_2012V1,2019-01-02,2018-12-27,A,B,-10864.3901,2014-12-25,A947,2014-12-30,1900-01-01,1900-01-01,NaN,P
5,5,6651310C,BUWP6,GNPP_2016V1,2019-01-02,2018-12-31,A,B,-221.3397,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,NaN,P
6,6,6651310C,RUWP6,GNPP_2016V1,2019-01-02,2018-12-31,A,B,-231.0452,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,NaN,P
7,7,6651310C,XUEAA,GNPP_2016V1,2019-01-02,2018-12-31,A,B,-212.5059,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,NaN,P
8,8,6651310C,XUEAA,GNPP_2016V1,2019-01-02,2018-12-31,A,B,433.2002,2017-01-01,A8D7,2016-12-28,1900-01-01,1900-01-01,NaN,P
9,9,6128381Y,SUEAM,XRP_2003V2,2019-01-03,2019-01-02,A,B,-327.4962,2004-11-01,E396,2004-10-22,1900-01-01,1900-01-01,NaN,P


In [17]:
%who_ls DataFrame 

[]

In [18]:
del df_All_Paxus_Switches
del df_All_Paxus_Switches_With_Product_Name
del df_All_Switches
del df_All_iSuite_Switches
del df_Product_Mapping
del df_iSuite_Product_list
del df_main

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-07-19 18:40:02.866570
svmem(total=16756752384, available=4362846208, percent=74.0, used=12393906176, free=4362846208)
